# Model Usage

`measurement_config.core` provides a Pydantic-based `Model` class that supports custom serialization for `numpy` and `tunits` objects.

In [1]:
from typing import Any

import numpy as np
import numpy.typing as npt

from measurement_config.core import Model
from measurement_config.quantities import Frequency
from measurement_config.units import GHz

## Defining a Model

You can define a model by inheriting from `Model`. Fields can include standard Python types, NumPy arrays, and `tunits` values.

In [2]:
from pydantic import Field


class ExperimentConfig(Model):
    """Configuration model for experiment parameters."""

    name: str
    frequency: Frequency
    pulse_shape: npt.NDArray[np.complexfloating]
    metadata: dict[str, Any] = Field(default_factory=dict)

In [3]:
config = ExperimentConfig(
    name="Rabi Oscillation",
    frequency=5.0 * GHz,
    pulse_shape=np.array([0.0 + 0.0j, 1.0 + 1.0j, 0.0 + 0.0j]),
    metadata={"some_value": 0.01},
)
print(config)

name='Rabi Oscillation' frequency=Frequency(5.0, 'GHz') pulse_shape=array([0.+0.j, 1.+1.j, 0.+0.j]) metadata={'some_value': 0.01}


## Serialization and Deserialization

The `Model` class handles serialization of special types automatically to a dictionary or JSON string.

Serialized output includes a top-level `__meta__` object with a `version` field for future format changes.

In [4]:
# To Dictionary
data_dict = config.to_dict()
print("Serialized dict:", data_dict)

# To JSON
json_str = config.to_json(indent=2)
print("JSON string:\n", json_str)

Serialized dict: {'name': 'Rabi Oscillation', 'frequency': {'units': [{'unit': 'HERTZ', 'scale': 'GIGA'}], 'real_value': 5.0, '__type__': 'tunits.Frequency'}, 'pulse_shape': {'complexes': {'values': [{'real': 0.0, 'imaginary': 0.0}, {'real': 1.0, 'imaginary': 1.0}, {'real': 0.0, 'imaginary': 0.0}]}, 'shape': [3], '__type__': 'numpy.ndarray'}, 'metadata': {'some_value': 0.01}, '__meta__': {'version': 0}}
JSON string:
 {
  "name": "Rabi Oscillation",
  "frequency": {
    "units": [
      {
        "unit": "HERTZ",
        "scale": "GIGA"
      }
    ],
    "real_value": 5.0,
    "__type__": "tunits.Frequency"
  },
  "pulse_shape": {
    "complexes": {
      "values": [
        {
          "real": 0.0,
          "imaginary": 0.0
        },
        {
          "real": 1.0,
          "imaginary": 1.0
        },
        {
          "real": 0.0,
          "imaginary": 0.0
        }
      ]
    },
    "shape": [
      3
    ],
    "__type__": "numpy.ndarray"
  },
  "metadata": {
    "some_valu

In [5]:
# From Dictionary/JSON
restored_config = ExperimentConfig.from_json(json_str)
print("Restored config:", restored_config)

# Verify equality
np.testing.assert_array_equal(config.pulse_shape, restored_config.pulse_shape)
assert config.frequency == restored_config.frequency

Restored config: name='Rabi Oscillation' frequency=Frequency(5.0, 'GHz') pulse_shape=array([0.+0.j, 1.+1.j, 0.+0.j]) metadata={'some_value': 0.01}
